In [0]:
!pip install --upgrade pip -q
!pip install progressbar -q
!pip install memory_profiler -q
!pip install --upgrade pandas>=1.2 -q

In [0]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
 %reload_ext memory_profiler

In [0]:
import urllib
import tarfile
import os
from collections import OrderedDict
import warnings
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import progressbar
from scipy.sparse import csr_matrix

from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.feature_extraction import FeatureHasher
from sklearn import linear_model
from sklearn.metrics import roc_auc_score, log_loss

## Download Criteo  Display Advertising Challenge dataset

In [0]:
# ProgressBar borrowed from https://stackoverflow.com/a/53643011/2015762
class ProgressBar():
    def __init__(self):
        self.pbar = None

    def __call__(self, block_num, block_size, total_size):
        if not self.pbar:
            self.pbar=progressbar.ProgressBar(maxval=total_size)
            self.pbar.start()

        downloaded = block_num * block_size
        if downloaded < total_size:
            self.pbar.update(downloaded)
        else:
            self.pbar.finish()


def download_dataset(dataset_url, dataset_folder_path, compressed_dataset_path):
    # Download dataset
    os.makedirs(dataset_folder_path, exist_ok=True)
    urllib.request.urlretrieve(dataset_url, compressed_dataset_path, ProgressBar())

def extract_dataset(compressed_dataset_path, dataset_folder_path, dataset_path):
    # Extract train.txt (dataset with labels) and readme
    with tarfile.open(compressed_dataset_path, "r") as input_file:
        input_file.extract('readme.txt', dataset_folder_path)
        input_file.extract('train.txt', dataset_folder_path)
        os.rename(os.path.join(dataset_folder_path, 'train.txt'), dataset_path)

In [0]:
dataset_url = "https://criteostorage.blob.core.windows.net/criteo-research-datasets/kaggle-display-advertising-challenge-dataset.tar.gz"
dataset_folder_path = os.path.abspath('sync/data/criteo_dataset')
compressed_dataset_path = os.path.join(dataset_folder_path, "criteo_dataset.tar.gz")
dataset_path = os.path.join(dataset_folder_path, "criteo_dataset.txt")

In [0]:
if not os.path.exists(compressed_dataset_path):
    download_dataset(dataset_url, dataset_folder_path, compressed_dataset_path)

if not os.path.exists(dataset_path):
    extract_dataset(compressed_dataset_path, dataset_folder_path, dataset_path)

If it takes too much time, download this smaller dataset instead

In [0]:
# toy_dataset_path = os.path.join(dataset_folder_path, "criteo_toy_dataset.txt")
# toy_dataset_url = 'https://www.dropbox.com/s/dle2t3szhljfevh/criteo_toy_dataset.txt?dl=1'
# urllib.request.urlretrieve(toy_dataset_url, toy_dataset_path, ProgressBar())

Quick look at the files we have downloaded.

Within iPython notebook, we can execute bash command by prepending the cell with `!` and insert python variable into it with `{}`

In [0]:
!ls -alh {dataset_folder_path}

total 16G
drwxr-xr-x 2 root root 4.0K Mar 12 15:20 .
drwxr-xr-x 3 root root 4.0K Mar 12 12:46 ..
-rw-r--r-- 1 root root 4.3G Mar 12 15:15 criteo_dataset.tar.gz
-rw-r--r-- 1 293604138 staff 11G May 12 2014 criteo_dataset.txt
-rw-r--r-- 1 root root 23M Mar 12 15:55 criteo_test_dataset.txt
-rw-r--r-- 1 root root 231M Mar 12 17:14 criteo_toy_dataset.txt
-rw-r--r-- 1 root root 208M Mar 12 15:55 criteo_train_dataset.txt
-rw-r--r-- 1 293604138 staff 1.9K Aug 22 2014 readme.txt

In [0]:
!cat {dataset_folder_path}/readme.txt

------ Display Advertising Challenge ------

Dataset: dac-v1

This dataset contains feature values and click feedback for millions of display 
ads. Its purpose is to benchmark algorithms for clickthrough rate (CTR) prediction.
It has been used for the Display Advertising Challenge hosted by Kaggle:
https://www.kaggle.com/c/criteo-display-ad-challenge/

===================================================

Full description:

This dataset contains 2 files:
 train.txt
 test.txt
corresponding to the training and test parts of the data. 

====================================================

Dataset construction:

The training dataset consists of a portion of Criteo's traffic over a period
of 7 days. Each row corresponds to a display ad served by Criteo and the first
column is indicates whether this ad has been clicked or not.
The positive (clicked) and negatives (non-clicked) examples have both been
subsampled (but at different rates) in order to reduce the dataset size.

There are 13 features taking integer values (mostly count features) and 26
categorical features. The values of the categorical features have been hashed
onto 32 bits for anonymization purposes. 
The semantic of these features is undisclosed. Some features may have missing values.

The rows are chronologically ordered.

The test set is computed in the same way as the training set but it 
corresponds to events on the day following the training period. 
The first column (label) has been removed.

====================================================

Format:

The columns are tab separeted with the following schema:
<label> <integer feature 1> ... <integer feature 13> <categorical feature 1> ... <categorical feature 26>

When a value is missing, the field is just empty.
There is no label field in the test set.

====================================================

Dataset assembled by Olivier Chapelle (o.chapelle@criteo.com)

In [0]:
label_columns = ['label']
integer_features = [f'int_feat_{i}' for i in range(1, 14)]
categorical_features = [f'cat_feat_{i}' for i in range(1, 27)]
columns = label_columns + integer_features + categorical_features

In [0]:
pd.read_csv(dataset_path, nrows=10, header=None, sep='\t', names=columns)

Out[84]:

,label,int_feat_1,int_feat_2,int_feat_3,int_feat_4,int_feat_5,int_feat_6,int_feat_7,int_feat_8,int_feat_9,int_feat_10,int_feat_11,int_feat_12,int_feat_13,cat_feat_1,cat_feat_2,cat_feat_3,cat_feat_4,cat_feat_5,cat_feat_6,cat_feat_7,cat_feat_8,cat_feat_9,cat_feat_10,cat_feat_11,cat_feat_12,cat_feat_13,cat_feat_14,cat_feat_15,cat_feat_16,cat_feat_17,cat_feat_18,cat_feat_19,cat_feat_20,cat_feat_21,cat_feat_22,cat_feat_23,cat_feat_24,cat_feat_25,cat_feat_26
0,0,1.0,1,5.0,0.0,1382,4.0,15,2,181,1.0,2,NaN,2.0,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102,8.0,2,2,4,1.0,1,NaN,4.0,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767,89.0,4,2,245,1.0,3,3.0,45.0,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392,NaN,0,0,0,NaN,0,NaN,NaN,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2,0.0,3,0,0,1.0,1,NaN,0.0,8cf07265,ae46a29d,c81688bb,f922efad,25c83c98,13718bbd,ad9fa255,0b153874,a73ee510,5282c137,e5d8af57,66a76a26,f06c53ac,1adce6ef,8ff4b403,01adbab4,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN
5,0,NaN,-1,NaN,NaN,12824,NaN,0,0,6,NaN,0,NaN,NaN,05db9164,6c9c9cf3,2730ec9c,5400db8b,43b19349,6f6d9be8,53b5f978,0b153874,a73ee510,3b08e48b,91e8fc27,be45b877,9ff13f22,07d13a8f,06969a20,9bc7fff5,776ce399,92555263,NaN,NaN,242bb710,8ec974f4,be7c41b4,72c78f11,NaN,NaN
6,0,NaN,1,2.0,NaN,3168,NaN,0,1,2,NaN,0,NaN,NaN,439a44a4,ad4527a2,c02372d0,d34ebbaa,43b19349,fe6b92e5,4bc6ffea,0b153874,a73ee510,3b08e48b,a4609aab,14d63538,772a00d7,07d13a8f,f9d1382e,b00d3dc9,776ce399,cdfa8259,NaN,NaN,20062612,NaN,93bad2c0,1b256e61,NaN,NaN
7,1,1.0,4,2.0,0.0,0,0.0,1,0,0,1.0,1,NaN,0.0,68fd1e64,2c16a946,503b9dbc,e4dbea90,f3474129,13718bbd,38eb9cf4,1f89b562,a73ee510,547c0ffe,bc8c9f21,60ab2f07,46f42a63,07d13a8f,18231224,e6b6bdc7,e5ba7672,74ef3502,NaN,NaN,5316a17f,NaN,32c7478e,9117a34a,NaN,NaN
8,0,NaN,44,4.0,8.0,19010,249.0,28,31,141,NaN,1,NaN,8.0,05db9164,d833535f,d032c263,c18be181,25c83c98,7e0ccccf,d5b6acf2,0b153874,a73ee510,2acdcf4e,086ac2d2,dfbb09fb,41a6ae00,b28479f6,e2502ec9,84898b2a,e5ba7672,42a2edb9,NaN,NaN,0014c32a,NaN,32c7478e,3b183c5c,NaN,NaN
9,0,NaN,35,NaN,1.0,33737,21.0,1,2,3,NaN,1,NaN,1.0,05db9164,510b40a5,d03e7c24,eb1fd928,25c83c98,NaN,52283d1c,0b153874,a73ee510,015ac893,e51ddf94,951fe4a9,3516f6e6,07d13a8f,2ae4121c,8ec71479,d4bb7bd8,70d0f5f9,NaN,NaN,0e63fca0,NaN,32c7478e,0e8fe315,NaN,NaN


## Reading data with memory constraints

We first create a toy dataset with "only" 1 million rows (out of 45 millions)

In [0]:
toy_dataset_path = os.path.join(dataset_folder_path, "criteo_toy_dataset.txt")

In [0]:
!head -n 1000000 {dataset_path} > {toy_dataset_path}

Let's say we want to perform a basic operation: estimate the number of positive samples within the data

### Basic approach

In [0]:
def compute_positive_label_proportion(dataset_path, columns):
    df = pd.read_csv(dataset_path, sep="\t", header=None, names=columns, usecols=['label'])
    return df['label'].mean()

Let's measure its memory footprint with the `%%memit` magic function

In [0]:
%%memit
positive_label_proportion = compute_positive_label_proportion(toy_dataset_path, columns)
print('positive_label_proportion', positive_label_proportion)

positive_label_proportion 0.254949
peak memory: 403.34 MiB, increment: 0.02 MiB

What would happen if you run the same function on a 45 times bigger dataset ?

You can give a try with `compute_positive_label_proportion(dataset_path, columns)`... at your own risks.

### Specifying column types
We can help pandas by specifying the column types to be used such that it does not need to infer it. Do so with the parameter dtype of pd.read_csv: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html.

In [0]:
col_types = OrderedDict()
for col_name in columns:
    if col_name in label_columns: col_type = 'bool'
    if col_name in integer_features: col_type = 'float32'
    if col_name in categorical_features: col_type = 'str'
    col_types[col_name] = col_type

def compute_positive_label_proportion_with_dtype(dataset_path, columns, col_types):
    # Read csv with dtype and return positive_label_proportion
    df = pd.read_csv(dataset_path, sep="\t", header=None, names=columns, dtype=col_types)
    return df['label'].mean()

In [0]:
%%memit
positive_label_proportion = compute_positive_label_proportion_with_dtype(toy_dataset_path, columns, col_types)
print('positive_label_proportion', positive_label_proportion)

positive_label_proportion 0.254949
peak memory: 1108.75 MiB, increment: 705.42 MiB

### Reading data by chunks
We can control the amount of memory we need by loading only a small chunk of the data and processing it before moving to the next chunk.

See documentation at https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html#iterating-through-files-chunk-by-chunk

```
reader = pd.read_csv(..., chunksize=10, nrows=100):
for chunk in reader:
    print(chunk)
```

In [0]:
def compute_positive_label_proportion_with_dtype_and_chunksize(dataset_path, columns, col_types, chunksize):
    # Read csv with dtype and chunksize and return positive_label_proportion
    reader = pd.read_csv(
        dataset_path, sep="\t", header=None, names=columns, dtype=col_types, chunksize=chunksize, 
    )
    sum_labels = 0
    sum_rows = 0
    for chunk in reader:
        sum_labels += chunk['label'].sum()
        sum_rows += len(chunk)
    return sum_labels / sum_rows

In [0]:
%%memit
positive_label_proportion = compute_positive_label_proportion_with_dtype_and_chunksize(toy_dataset_path, columns, col_types, 100_000)
print('positive_label_proportion', positive_label_proportion)

positive_label_proportion 0.254949
peak memory: 471.07 MiB, increment: 43.71 MiB

This can now be applied to the full dataset with no memory issue.

In [0]:
%%memit
positive_label_proportion = compute_positive_label_proportion_with_dtype_and_chunksize(dataset_path, columns, col_types, 100_000)
print('positive_label_proportion', positive_label_proportion)

## Training and evaluation

### Split train and test datasets
Since the datasets contain one line per example, we can split them into train and test by simply iterating over the lines. For each line in the original dataset: write it to the test data set with a probability p and write it to the train dataset with a probability 1 - p.

In [0]:
def split_train_test(full_dataset_path, train_dataset_path, test_dataset_path, test_ratio, seed=302984, print_every=None):
    random.seed(seed)
    with open(full_dataset_path, 'r') as input_file, open(train_dataset_path, 'w') as train_file, open(test_dataset_path, 'w') as test_file:
        for i, line in enumerate(input_file):
            if random.uniform(0, 1) <= test_ratio:
                test_file.write(line)
            else:
                train_file.write(line)
            
            if print_every is not None and (i + 1) % print_every == 0:
                print(f"Processed {i + 1} lines")
        print(f"Processed {i + 1} lines")
        
train_dataset_path = os.path.join(dataset_folder_path, "criteo_train_dataset.txt")
test_dataset_path = os.path.join(dataset_folder_path, "criteo_test_dataset.txt")

In [0]:
if not os.path.exists(train_dataset_path) or not os.path.exists(test_dataset_path):
    split_train_test(dataset_path, train_dataset_path, test_dataset_path, test_ratio=0.1, print_every=10_000_000)

Processed 10000000 lines
Processed 20000000 lines
Processed 30000000 lines
Processed 40000000 lines
Processed 45840617 lines

In [0]:
!wc -l {test_dataset_path}

4585250 /databricks/driver/sync/data/criteo_dataset/criteo_test_dataset.txt

### Shuffling
The convergence guarantees of SGD rely on the fact that the observations come at random. Hence, shuffling between epochs is important.

First result of "How to shuffle a file that is too big for memory" on Google: https://stackoverflow.com/a/40814865/2015762

Note that quicker pseudo-shuffling strategies exists, but this fits our "Big data on your laptop" problematic.

In [0]:
!awk 'BEGIN{srand();} {printf "%06d %s\n", rand()*1000000, $0;}' /databricks/driver/sync/data/criteo_dataset/criteo_test_dataset.txt | sort -n | cut -c8- > /databricks/driver/sync/data/criteo_dataset/criteo_test_dataset_shuffled.txt
# We can run it on the train dataset too but let'ss skip it since it is quite long
# !awk 'BEGIN{srand();} {printf "%06d %s\n", rand()*1000000, $0;}' /databricks/driver/sync/data/criteo_dataset/criteo_train_dataset.txt | sort -n | cut -c8- > /databricks/driver/sync/data/criteo_dataset/criteo_train_dataset_shuffled.txt

### Training
In order to train a logistic model on chunks of data, we will use scikit-learn `SGDClassifier` (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) and train for its `log` loss with its `partial_fit` method.
We can now apply the previous data processing pipeline and add the training to obtain a trained classifier.

In [0]:
#  To begin with, let's not do any preprocessing and deal with "ready to use" continuous features only
def preprocess_data(chunk, integer_features, categorical_features):
    return chunk[integer_features].fillna(-1)

In [0]:
max_training_steps = 10_000
chunk_size = 1_000
print_every = 1000

classifier = SGDClassifier(loss="log")
# 1. Read train data by chunks
reader = pd.read_csv(
    train_dataset_path, sep="\t", header=None, names=columns, dtype=col_types, chunksize=chunk_size, 
)
for i, chunk in enumerate(reader):
    # 2. Apply preprocess_data to return the continous features
    features = preprocess_data(chunk, integer_features, categorical_features)
    # 3. Train classifier on this chunk  with fit.
    classifier.partial_fit(features, chunk["label"], classes=[0, 1])
    # 4. Stop after `max_training_steps`
    if i > max_training_steps:
        break
        
    if print_every is not None and (i + 1) % print_every == 0:
        print(i+1)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000

### Testing
Let's evaluate the performances of the trained classifier. We should iterate over the test dataset and evaluate the labels predicted by the classifier with `roc_auc_score` and `log_loss`.

In [0]:
max_testing_steps = 1_000
chunk_size = 1_000
print_every = 100

reader = pd.read_csv(
    test_dataset_path, sep="\t", header=None, names=columns, dtype=col_types, chunksize=chunk_size, 
)

roc_auc_scores = []
log_losses = []
# 1. Read test data by chunks
for i, chunk in enumerate(reader):
    # 2. Apply preprocess_data to return the continous features
    features = preprocess_data(chunk, integer_features, categorical_features)
    # 3. Predict labels with classifiers
    label_predictions = classifier.predict_proba(features)[:, 1]
    # 4. Compute AUC score and Log loss for this chunk
    roc_auc_scores += [roc_auc_score(chunk["label"], label_predictions)]
    log_losses += [log_loss(chunk["label"], label_predictions)]
    
    if i > max_testing_steps:
        break
        
    if print_every is not None and (i + 1) % print_every == 0:
        print(i+1)

# 6. Return averaged values of the metrics
print(f"AUC = {np.mean(roc_auc_scores)}")
print(f"LogLoss = {np.mean(log_losses)}")

100
200
300
400
500
600
700
800
900
1000
AUC = 0.7614335822071854
LogLoss = 0.4867119718778749

## Data preprocessing

### Continuous features
A smart way to deal with continuous features (counting integer features are part of them), consists in transforming them into categorical features through a quantile transformation. To do so we will use scikit-learn KBinsDiscretizer : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html.

It can be used as following
```
df = pd.DataFrame({'col_1': np.random.normal(size=1000), 'col_2': np.random.poisson(lam=1, size=1000)})
bucketizer = KBinsDiscretizer(n_bins=20, encode='ordinal')
bucketizer.fit(df)
df_bucketized = pd.DataFrame(bucketizer.transform(df), columns=[f'{col}_bucketized' for col in df.columns], index=df.index)
sns.jointplot(data=pd.concat((df, df_bucketized), axis=1), x="col_1", y="col_1_bucketized")
```

1. Create a `KBinsDiscretizer` and train it on the first chunk of the dataset
1. Update `preprocess_data` to add a bucketize step to the training pipeline. What happens if you change the `encode` parameter?
1. Do not forget to deal with missing values, you do not want to carry on NaNs. You can for example replace them with -1.

In [0]:
bucketizer = KBinsDiscretizer(n_bins=20) # Try with and without encode='ordinal'
with warnings.catch_warnings(record=True):
    bucketizer.fit(chunk[integer_features].fillna(-1))
    
def bucketize(df, bucketizer):
    return bucketizer.transform(df.fillna(-1))

def preprocess_data(chunk, integer_features, categorical_features):
    return bucketize(chunk[integer_features], bucketizer)

preprocess_data(chunk, integer_features, categorical_features)

Out[104]: <1000x166 sparse matrix of type '<class 'numpy.float64'>'
	with 13000 stored elements in Compressed Sparse Row format>

### Categorical features
For categorical features we will implement the hashing trick by ourselves. As a quick reminder, for each row

1. Select the categorical features 
1. Create for each feature the string concatenating the feature name and the feature value
1. Apply a hash function to each of these string and use this value to choose the feature's column index
1. Store the transformed features in a sparse matrix

In [0]:
from sklearn.utils.murmurhash import murmurhash3_bytes_s32

def hash_string(string, seed=0):
    return murmurhash3_bytes_s32(string.encode(), seed)
  
hash_string('my_feature=my_feature_value')
# Note, if we were using builtin function hash('my_feature=my_feature_value'), we would have had a different hash value at each run

Out[106]: 1480568101

In [0]:
hash_space = 2 ** 16

row = chunk.iloc[0]
def get_features_hashes(row, hash_space):
    # return the list of the hashes values for each categorical feature in the row
    features_as_string = [f"{label}={value}" for label, value in zip(row.index, row.values)]
    # https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/feature_extraction/_hashing_fast.pyx#L68
    return [abs(hash_string(string)) % hash_space for string in features_as_string]

np.array(get_features_hashes(row, hash_space))

Out[107]: array([26837, 21638, 49088, 37471, 7692, 41062, 24334, 64092, 5751,
 5424, 40976, 40268, 52632, 11692, 34626, 45899, 20384, 11307,
 21604, 23634, 61337, 53234, 11512, 22037, 26718, 58424, 45763,
 10187, 35706, 51356, 29342, 6149, 25889, 53956, 35379, 12896,
 45210, 53805, 28145, 30177])

In [0]:
def transform_with_hashing_trick(df, hash_space):
    col_indices = df.apply(lambda row: get_features_hashes(row, hash_space), axis=1) 
    row_indices = [
        [row_index] * len(cols)
        for row_index, cols in zip(np.arange(len(col_indices)), col_indices)
    ]

    flat_col_indices = sum(col_indices.values, [])
    flat_row_indices = sum(row_indices, [])
    data = np.ones_like(flat_col_indices)
    # Fill the csr_matrix, using csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)]) constructor
    # See https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
    return csr_matrix((data, (flat_row_indices, flat_col_indices)), shape=(len(df), hash_space), dtype=float)
  
transform_with_hashing_trick(chunk[categorical_features], hash_space)

Out[108]: <1000x65536 sparse matrix of type '<class 'numpy.float64'>'
	with 25998 stored elements in Compressed Sparse Row format>

Actually, the hashing trick is well known and already implemented in scikit-learn FeatureHasher: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html

It can be used as following
```
df = pd.DataFrame({'col_1': np.random.choice(['a', 'b', 'c'], size=100), 'col_2': np.random.poisson(size=100)})
hasher = FeatureHasher(n_features=2**16, input_type="dict")
hasher.transform((row._asdict() for row in df.itertuples(index=False)))
```
Again, you will probably want to ensure you get rid of the NaNs. What value could you set for these?

In [0]:
hasher = FeatureHasher(n_features=hash_space, input_type="dict")

def feature_hashing(df, hasher):
    # apply hasher.transorm to rows of df
    return hasher.transform((row._asdict() for row in df.fillna("nan").itertuples(index=False)))

We can compare their speed

In [0]:
%timeit transform_with_hashing_trick(chunk[categorical_features], hash_space)

186 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

In [0]:
%timeit feature_hashing(chunk[categorical_features], hasher)

28.3 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Actually we have used exactly the same implementation as sklearn.
See https://github.com/scikit-learn/scikit-learn/blob/95119c13af77c76e150b753485c662b7c52a41a2/sklearn/feature_extraction/_hashing_fast.pyx#L68

In [0]:
hash_space = 20
df = chunk[categorical_features].head(5)
print('Custom implementation')
print(transform_with_hashing_trick(df, hash_space).toarray())
hasher = FeatureHasher(n_features=hash_space, input_type="dict", alternate_sign=False)
print('sklearn implementation')
print(feature_hashing(df, hasher).toarray())

Custom implementation
[[1. 2. 3. 1. 2. 1. 1. 2. 1. 2. 0. 2. 1. 1. 1. 0. 2. 1. 2. 0.]
 [4. 1. 2. 3. 1. 0. 0. 0. 1. 1. 1. 2. 1. 0. 3. 0. 0. 1. 2. 3.]
 [2. 1. 0. 2. 2. 0. 0. 1. 1. 0. 3. 1. 2. 1. 2. 2. 1. 1. 3. 1.]
 [0. 3. 2. 4. 1. 1. 3. 0. 1. 1. 1. 2. 2. 1. 0. 2. 0. 1. 0. 1.]
 [2. 1. 3. 2. 1. 2. 2. 0. 1. 2. 2. 1. 1. 0. 1. 1. 1. 1. 0. 2.]]
sklearn implementation
[[1. 2. 3. 1. 2. 1. 1. 2. 1. 2. 0. 2. 1. 1. 1. 0. 2. 1. 2. 0.]
 [4. 1. 2. 3. 1. 0. 0. 0. 1. 1. 1. 2. 1. 0. 3. 0. 0. 1. 2. 3.]
 [2. 1. 0. 2. 2. 0. 0. 1. 1. 0. 3. 1. 2. 1. 2. 2. 1. 1. 3. 1.]
 [0. 3. 2. 4. 1. 1. 3. 0. 1. 1. 1. 2. 2. 1. 0. 2. 0. 1. 0. 1.]
 [2. 1. 3. 2. 1. 2. 2. 0. 1. 2. 2. 1. 1. 0. 1. 1. 1. 1. 0. 2.]]

Let's improve our previous pipeline and apply the hashing trick to the categorical features **and** to the bucketized continuous features (have a look at `pd.concat`).

In [0]:
hash_space = 2 ** 20

hasher = FeatureHasher(n_features=hash_space, input_type="dict")
bucketizer = KBinsDiscretizer(n_bins=20, encode='ordinal')
with warnings.catch_warnings(record=True):
    bucketizer.fit(chunk[integer_features].fillna(-1))

def bucketize(df, bucketizer):
    return pd.DataFrame(bucketizer.transform(df.fillna(-1)), columns=df.columns, index=df.index)

def preprocess_data(df, integer_features, categorical_features):
    bucketized_integer_features_df = bucketize(chunk[integer_features], bucketizer)
    categorical_features_df = pd.concat([chunk[categorical_features], bucketized_integer_features_df], axis=1)
    return feature_hashing(categorical_features_df, hasher)

preprocess_data(chunk, integer_features, categorical_features)

Out[121]: <1000x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 36135 stored elements in Compressed Sparse Row format>

Going further with the hashing trick:
1. How could you implement cross features ?
1. If you are too afraid of collisions, try to hash each categorical feature to several locations.

Try with FeatureHasher and your custom implementation and see what is the most efficient.